In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df = pd.read_csv('Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,27783.000000,27783.000000,27783.000000,27783.000000,2.778300e+04
mean,13892.000000,1.564626,2.014073,4.145485,1.294322e+09
std,8020.405601,5.327324,5.937523,1.318098,4.707710e+07
min,1.000000,0.000000,0.000000,1.000000,9.617184e+08
25%,6946.500000,0.000000,0.000000,4.000000,1.268136e+09
50%,13892.000000,0.000000,1.000000,5.000000,1.307837e+09
75%,20837.500000,1.000000,2.000000,5.000000,1.329696e+09
max,27783.000000,202.000000,219.000000,5.000000,1.351210e+09


In [ ]:
df.isna().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               1
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

In [ ]:
df.dropna(inplace=True)
df.isna().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

In [ ]:
df['Score'].value_counts()

5    17221
4     4070
1     2538
3     2340
2     1613
Name: Score, dtype: int64

In [ ]:
def apply_sentiment(Score):
    if(Score <=3):
        return 0
    else:
        return 1

In [ ]:
df['Sentiment'] = df['Score'].apply(apply_sentiment)

In [ ]:
data = pd.DataFrame()
data['ProductId'] = df['ProductId']
data['Sentiment'] = df['Sentiment']
data['Review'] = df['Text']
data.head()

,ProductId,Sentiment,Review
0,B001E4KFG0,1,I have bought several of the Vitality canned d...
1,B00813GRG4,0,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,1,This is a confection that has been around a fe...
3,B000UA0QIQ,0,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,1,Great taffy at a great price. There was a wid...


In [ ]:
data['Sentiment'].value_counts()

1    21291
0     6491
Name: Sentiment, dtype: int64

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def clean_text(Review):
    Review = str(Review).lower() # convert to lowercase
    Review = re.sub('\[.*?\]', '', Review) 
    Review = re.sub('https?://\S+|www\.\S+', '', Review) # Remove URls
    Review = re.sub('<.*?>+', '', Review)
    Review = re.sub(r'[^a-z0-9\s]', '', Review) # Remove punctuation
    Review = re.sub('\n', '', Review)
    Review = re.sub('\w*\d\w*', '', Review)
    Review = [lemmatizer.lemmatize(word) for word in Review.split() if not word in set(stopwords.words('english'))]
    Review = ' '.join(Review)
    return Review

In [ ]:
data['Review'] = data['Review'].apply(clean_text)

In [ ]:
data.head()

,ProductId,Sentiment,Review
0,B001E4KFG0,1,bought several vitality canned dog food produc...
1,B00813GRG4,0,product arrived labeled jumbo salted peanutsth...
2,B000LQOCH0,1,confection around century light pillowy citrus...
3,B000UA0QIQ,0,looking secret ingredient robitussin believe f...
4,B006K2ZZ7K,1,great taffy great price wide assortment yummy ...


In [ ]:
data.to_csv("Amazon_Review_Clean.csv")

In [ ]:
X = data['Review']
y = data['Sentiment']


In [ ]:
y

0        1
1        0
2        1
3        0
4        1
        ..
27778    1
27779    1
27780    1
27781    1
27782    1
Name: Sentiment, Length: 27782, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.25,random_state=42)

In [ ]:
y_test

array([1, 0, 1, ..., 1, 1, 1])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(use_idf=True, tokenizer=word_tokenize)
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(20836,)
(6946,)


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_seq_padded = pad_sequences(X_train_seq, maxlen=64)
X_test_seq_padded = pad_sequences(X_test_seq, maxlen=64)
X_train_seq_padded[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
         218,    25,    29,   201,  1919,   482,    78,   144,   107,
         274,    85, 15518,  1155,   247,   273,   470,  3391,   470,
       15519], dtype=int32)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.index_word)+1,64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 64)          2449344   
                                                                 
 bidirectional_4 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,478,401
Trainable params: 2,478,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor="val_loss",patience=3,verbose=True)

In [ ]:
history = model.fit(X_train_seq_padded, y_train, epochs=10,
                    validation_data=(X_test_seq_padded, y_test),callbacks=[early_stop])

Epoch 1/10
652/652 [==============================] - 58s 82ms/step - loss: 0.3755 - accuracy: 0.8372 - val_loss: 0.3304 - val_accuracy: 0.8615
Epoch 2/10
652/652 [==============================] - 52s 79ms/step - loss: 0.2230 - accuracy: 0.9140 - val_loss: 0.3580 - val_accuracy: 0.8624
Epoch 3/10
652/652 [==============================] - 52s 80ms/step - loss: 0.1336 - accuracy: 0.9515 - val_loss: 0.4046 - val_accuracy: 0.8563
Epoch 4/10
652/652 [==============================] - 66s 101ms/step - loss: 0.0802 - accuracy: 0.9725 - val_loss: 0.5073 - val_accuracy: 0.8462
Epoch 4: early stopping


In [52]:
from sklearn.metrics import roc_auc_score
pred_train = model.predict(X_train_seq_padded)
pred_test = model.predict(X_test_seq_padded)
print('Training Score: ' + str(roc_auc_score(y_train, pred_train)))
print('Testing Score: ' + str(roc_auc_score(y_test, pred_test)))

Training Score: 0.9983825568243526
Testing Score: 0.8739125791629874


In [44]:
y_test

array([1, 0, 1, ..., 1, 1, 1])

In [54]:
pred_test = np.where(pred_test > 0.5, 1, 0)

In [55]:
pred_test

array([[1],
       [0],
       [1],
       ...,
       [1],
       [0],
       [0]])

In [56]:
from sklearn.metrics import accuracy_score
print("Accuracy Score")
print(accuracy_score(pred_test, y_test))

Accuracy Score
0.8462424416930607


In [58]:
print(pred_test[0:24].reshape( 1, -1))
print(y_test[0:24])

[[1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1]]
[1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1]
